<a href="https://colab.research.google.com/github/huiesoo/esaa/blob/main/24-1/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B81_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 패키지

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

## 데이터

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ESAA/24-1/235745_parking_data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ESAA/24-1/235745_parking_data/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/ESAA/24-1/235745_parking_data/sample_submission.csv')

## 에러 처리

In [ ]:
# train_error = ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
# train = train[~train['단지코드'].isin(train_error)].reset_index(drop=True)
# test_error = ['C2675', 'C2335', 'C1327']
# test = test[~test['단지코드'].isin(test_error)].reset_index(drop=True)

In [ ]:
train.shape, test.shape

((2952, 15), (1022, 14))

In [ ]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [ ]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


## 결측치 확인

In [ ]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

In [ ]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

train -> 임대보증금, 임대료, 지하철역 수, 버스정류장 수  
test -> 임대보증금, 임대료, 지하철역 수, 자격유형

In [ ]:
# 지하철역 수, 버스정류장 수 NULL -> 0으로 대체
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

In [ ]:
test[test.자격유형.isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [ ]:
# 단지코드 C2411면 자격유형 모두 A -> A로 대체
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'

# 단지코드 C2253면 임대보증금과 임대료가 존재하는 경우 자격유형 C -> C로 대체
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

## 칼럼 확인

In [ ]:
train.groupby(['단지코드']).nunique(dropna=False).sum(axis=0)

총세대수                             411
임대건물구분                           444
지역                               411
공급유형                             475
전용면적                            1835
전용면적별세대수                        2155
공가수                              411
자격유형                             498
임대보증금                           1234
임대료                             1248
도보 10분거리 내 지하철역 수(환승노선 수 반영)     411
도보 10분거리 내 버스정류장 수               411
단지내주차면수                          411
등록차량수                            411
dtype: int64

**하나의 단지코드에 하나의 값만 존재** : 총세대수, 지역, 공가수, 지하철역 수, 버스정류장 수, 단지내주차면수, 등록차량수

**두 개 이상의 값 존재** : 임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증금, 임대료

## 지역명 숫자로 매핑

In [ ]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i

train['지역'] = train['지역'].map(local_map)
test['지역'] = test['지역'].map(local_map)

In [ ]:
# 하나의 값만 존재하는 칼럼만 우선 데이터셋으로 만들기

unique_cols = ['총세대수', '지역', '공가수',
               '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
               '도보 10분거리 내 버스정류장 수',
               '단지내주차면수', '등록차량수']
train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
test_agg = test.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()

## 1) 임대건물구분
아파트 or 상가 - OneHot인코딩

In [ ]:
def reshape_cat_features(data, cast_col, value_col):
    res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
    res.columns.name = None
    res = res.rename(columns={col:cast_col+'_'+col for col in res.columns})
    return res

In [ ]:
reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter')
reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter')

,임대건물구분_상가,임대건물구분_아파트
단지코드,,
C1003,0.0,1.0
C1006,1.0,1.0
C1016,0.0,1.0
C1019,0.0,1.0
C1030,0.0,1.0
...,...,...
C2653,0.0,1.0
C2675,0.0,1.0
C2676,1.0,1.0



## 2) 공급유형과 자격유형

공급유형이 대부분의 자격유형 데이터 설명 가능  
-> 자격유형 변수는 사용하지 않음, 공급유형은 OneHot인코딩

In [ ]:
pd.crosstab(train['공급유형'], train['자격유형'])

자격유형,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
공급유형,,,,,,,,,,,,,,,
공공분양,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0
공공임대(10년),205,0,0,0,0,0,0,0,0,0,0,0,0,0,0
공공임대(50년),31,0,0,0,0,0,0,0,0,0,0,0,0,0,0
공공임대(5년),3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
공공임대(분납),12,0,0,0,0,0,0,0,0,0,0,0,0,0,0
국민임대,1539,21,0,0,34,0,9,155,0,0,0,0,0,0,0
영구임대,2,0,95,0,3,3,0,0,49,0,0,0,0,0,0
임대상가,0,0,0,562,0,0,0,0,0,0,0,0,0,0,0
장기전세,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
reshape_cat_features(data=train, cast_col='공급유형', value_col='counter')

,공급유형_공공분양,공급유형_공공임대(10년),공급유형_공공임대(50년),공급유형_공공임대(5년),공급유형_공공임대(분납),공급유형_국민임대,공급유형_영구임대,공급유형_임대상가,공급유형_장기전세,공급유형_행복주택
단지코드,,,,,,,,,,
C1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C1004,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
C1005,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C1013,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C1014,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
C2663,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C2666,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C2670,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## 3) 전용면적
단지코드별 mean, 전체 sum 변수로 사용

In [ ]:
train.groupby(['단지코드'])['전용면적'].mean()

단지코드
C1000    45.218571
C1004    39.136842
C1005    54.360000
C1013    46.142000
C1014    49.232857
           ...    
C2663    41.005000
C2666    43.258333
C2670    40.918000
C2680    43.350000
C2692    43.570000
Name: 전용면적, Length: 423, dtype: float64

In [ ]:
면적_mean = train.groupby(['단지코드'])['전용면적'].mean()
면적_sum = train.groupby(['단지코드'])['전용면적'].sum()
train_agg['전용면적 평균'] = 면적_mean
train_agg['전용면적 총합'] = 면적_sum

면적_mean = test.groupby(['단지코드'])['전용면적'].mean()
면적_sum = test.groupby(['단지코드'])['전용면적'].sum()
test_agg['전용면적 평균'] = 면적_mean
test_agg['전용면적 총합'] = 면적_sum

## 4) 전용면적 별 세대수, 임대보증금, 임대료
드랍



---



## 범주형 인코딩

In [ ]:
# 아까 만들어둔 train_agg에 병합
train_agg = pd.concat([train_agg,
                       reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='공급유형', value_col='counter')
                       ], axis=1)

test_agg = pd.concat([test_agg,
                       reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='공급유형', value_col='counter')
                       ], axis=1)



---



## 이상치 처리

In [ ]:
train_agg['주차면수/총세대수'] = train_agg['단지내주차면수'] / train_agg['총세대수']

Q1 = train_agg['주차면수/총세대수'].quantile(0.25)
Q3 = train_agg['주차면수/총세대수'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = train_agg[(train_agg['주차면수/총세대수'] < lower_bound) | (train_agg['주차면수/총세대수'] > upper_bound)].index
train_agg = train_agg.drop(outliers, axis=0)

train_agg = train_agg.drop('주차면수/총세대수', axis=1)

# test 가공

In [ ]:
# test에 없는 칼럼 채우기
test_agg['공급유형_공공임대(5년)']=0
test_agg['공급유형_장기전세']=0
test_agg['공급유형_공공분양']=0

In [ ]:
# 칼럼 순서 맞춰주기
feature_order = ['총세대수', '지역', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수', '전용면적 평균', '전용면적 총합', '임대건물구분_상가', '임대건물구분_아파트', '공급유형_공공분양', '공급유형_공공임대(10년)', '공급유형_공공임대(50년)', '공급유형_공공임대(5년)', '공급유형_공공임대(분납)', '공급유형_국민임대', '공급유형_영구임대', '공급유형_임대상가', '공급유형_장기전세', '공급유형_행복주택']
test_agg = test_agg[feature_order]

# Modeling
Catboost, Lasso, RandomForest, Xgboost

In [ ]:
X_train = train_agg.drop(['등록차량수'], axis = 1)
y_train = train_agg['등록차량수']
X_test = test_agg

In [ ]:
train_agg.corr()['등록차량수'].drop('등록차량수').sort_values(ascending=False)

단지내주차면수                         0.848199
총세대수                            0.579290
공급유형_공공임대(10년)                  0.405082
전용면적 평균                         0.358033
공급유형_공공분양                       0.217199
공급유형_공공임대(분납)                   0.194895
공가수                             0.167420
도보 10분거리 내 버스정류장 수              0.108576
공급유형_장기전세                       0.096647
전용면적 총합                         0.068075
공급유형_국민임대                       0.052240
공급유형_공공임대(5년)                   0.039219
지역                              0.032305
도보 10분거리 내 지하철역 수(환승노선 수 반영)   -0.013439
공급유형_공공임대(50년)                 -0.110283
공급유형_행복주택                      -0.170489
공급유형_영구임대                      -0.204401
임대건물구분_상가                      -0.315019
공급유형_임대상가                      -0.315019
임대건물구분_아파트                           NaN
Name: 등록차량수, dtype: float64

In [ ]:
X_train = train_agg.drop(['등록차량수', '임대건물구분_상가', '임대건물구분_아파트'], axis = 1)
y_train = train_agg['등록차량수']
X_test = test_agg.drop(['임대건물구분_상가', '임대건물구분_아파트'], axis=1)

### 스케일링

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler

scailing_features = ['총세대수', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수', '전용면적 평균', '전용면적 총합']

scaler = StandardScaler()
X_train.loc[:, scailing_features] = scaler.fit_transform(X_train[scailing_features])
X_test.loc[:, scailing_features] = scaler.transform(X_test[scailing_features])

### 1) CatBoost

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.0 MB/s eta 0:00:00


In [ ]:
# 하이퍼 파라미터 튜닝
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

cat = CatBoostRegressor(random_state=3)

param_grid = {
    # 'n_estimators' : [100, 150, 200]
    'iterations' : [500],
    # 'learning_rate': [0.01, 0.05, 0.1]
    'depth': [2, 4, 6, 8]
    # 'l2_leaf_reg': [1, 3, 5]
    # 'bagging_temperature': [0.0, 1.0, 5.0]
}

gridcv = GridSearchCV(cat, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
gridcv.fit(X_train, y_train)

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
10:	learn: 294.7047938	total: 17.4ms	remaining: 772ms
11:	learn: 288.2239474	total: 19ms	remaining: 774ms
12:	learn: 280.5319382	total: 20.6ms	remaining: 771ms
13:	learn: 273.9333217	total: 22.1ms	remaining: 768ms
14:	learn: 268.5887501	total: 23.8ms	remaining: 768ms
15:	learn: 262.8563156	total: 25.3ms	remaining: 766ms
16:	learn: 256.6615002	total: 27.1ms	remaining: 769ms
17:	learn: 251.5717265	total: 28.6ms	remaining: 767ms
18:	learn: 245.8889286	total: 30.2ms	remaining: 766ms
19:	learn: 240.4212725	total: 31.8ms	remaining: 764ms
20:	learn: 235.7698104	total: 33.4ms	remaining: 763ms
21:	learn: 231.1810605	total: 35ms	remaining: 760ms
22:	learn: 227.2920937	total: 36.5ms	remaining: 758ms
23:	learn: 223.1329767	total: 38.1ms	remaining: 756ms
24:	learn: 218.9289427	total: 39.7ms	remaining: 754ms
25:	learn: 215.6573637	total: 40.9ms	remaining: 746ms
26:	learn: 212.1216965	total: 42.6ms	remaining: 747ms
27:	learn: 208.8042092	total: 44.3ms	remaining: 74

In [ ]:
# 최적 파라미터로 훈련
from catboost import CatBoostRegressor
cat = CatBoostRegressor(n_estimators=100, depth=3, random_state=600)
cat.fit(X_train, y_train)

In [ ]:
cat = CatBoostRegressor(learning_rate=0.01, depth=6, l2_leaf_reg=1, bagging_temperature=0, random_state=3)
cat.fit(X_train, y_train)

In [ ]:
cat = CatBoostRegressor(n_estimators=100, learning_rate=0.1, depth=4, l2_leaf_reg=3, bagging_temperature=0, random_state=3)
cat.fit(X_train, y_train)

In [ ]:
cat = CatBoostRegressor(n_estimators=200, random_state=3)
cat.fit(X_train, y_train)

In [ ]:
cat = CatBoostRegressor(n_estimators=300, depth=2)
cat.fit(X_train, y_train)

In [ ]:
cat = CatBoostRegressor(iterations=1000, depth=2, random_state=3)
cat.fit(X_train, y_train)

In [ ]:
y_pred = cat.predict(X_test)

In [ ]:
# submission = submission[~submission['code'].isin(test_error)].reset_index(drop=True)
submission['num'] = y_pred
# error = pd.DataFrame(np.array([['C2675', 0], ['C2335', 0], ['C1327', 0]]), columns=['code', 'num'])
# submission = pd.concat([submission, error]).reset_index(drop=True)
submission.to_csv('/content/drive/MyDrive/ESAA/24-1/235745_parking_data/submission_cat8.csv', index=False)

리더보드 : 96.9261665811 // 105.7249073475  
(로그변환 O : 105.969747828 // 107.3728771102)

### 2) Lasso

In [ ]:
# 하이퍼 파라미터 튜닝
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso = Lasso(random_state=3)

params = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'max_iter': [100, 500, 1000]
}

gridcv = GridSearchCV(lasso, param_grid=params, cv=5)
gridcv.fit(X_train, y_train)

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.878e+06, tolerance: 5.094e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.839e+06, tolerance: 5.654e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.283e+06, tolerance: 5.824e

GridSearchCV 최적 파라미터: {'alpha': 100, 'max_iter': 100}


In [ ]:
# 최적 파라미터로 훈련

lasso = Lasso(alpha=0.4, max_iter=300)
lasso.fit(X_train, y_train)

Lasso(alpha=0.4, max_iter=300)

In [ ]:
lasso = Lasso(alpha=100, max_iter=100)
lasso.fit(X_train, y_train)

Lasso(alpha=100, max_iter=100)

In [ ]:
y_pred = lasso.predict(X_test)

In [ ]:
submission['num'] = y_pred
submission.to_csv('/content/drive/MyDrive/ESAA/24-1/235745_parking_data/submission_lasso2.csv', index=False)

리더보드 : 115.825145762 // 114.7795181507

### 3) XGBoost

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(random_state=3)

params = {'min_child_weight':[0.1,0.3,0.5],
          'max_depth':[1,3,5],
          'subsample':[0.1,0.3,0.5],
          'n_estimators': [100,300]
}

gridcv = GridSearchCV(xgb, param_grid=params, cv=5)
gridcv.fit(X_train, y_train)

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

GridSearchCV 최적 파라미터: {'max_depth': 1, 'min_child_weight': 0.1, 'n_estimators': 100, 'subsample': 0.5}


In [ ]:
# 최적 파라미터로 훈련

xgb = XGBRegressor(max_depth=1, min_child_weight=0.1, n_estimators=100, subsample=0.5)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=1, max_leaves=None,
             min_child_weight=0.1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = xgb.predict(X_test)

In [ ]:
submission['num'] = y_pred
submission.to_csv('/content/gdrive/MyDrive/ESAA/data/parking/submission_xgb.csv', index=False)

리더보드 : 105.6472499633 // 114.1366248878

### 4) Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=3)

params = {'n_estimators': [100,300,500],
              'max_depth': [5, 10, 20],
              'min_samples_split': [5,10,15],
              'min_samples_leaf': [1, 2, 4]
}
gridcv = GridSearchCV(rf, param_grid=params, cv=5)
gridcv.fit(X_train, y_train)

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

GridSearchCV 최적 파라미터: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 100}


In [ ]:
rf = RandomForestRegressor(max_depth=5, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, min_samples_leaf=4, min_samples_split=5)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
submission['num'] = y_pred
submission.to_csv('/content/gdrive/MyDrive/ESAA/data/parking/submission_rf.csv', index=False)

리더보드 : 123.1535765399 // 116.9883346041